# This section is to see if there were any significant drops in spots for certain hybs.

In [ ]:
import matplotlib.pyplot  as plt
import scipy.stats as st
import pandas as pd
from util import pil_imread
import numpy as np

%config InlineBackend.figure_format = 'retina'

In [ ]:
counts_pos = []
for pos in range(50):
    try:
        spots = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/230608_4k_inv_5bs/pyfish_tools/output/dots_detected/Channel_All/Pos{pos}/noise_removed_locations_z_0.csv")
        num_cells = len(spots["cell number"].unique())
    except:
        continue
    counts = [len(spots[spots.hyb == hyb])/num_cells for hyb in np.sort(spots.hyb.unique())]
    if counts[0] < 1000:
        continue
    counts_pos.append(counts)

In [ ]:
counts_df = pd.DataFrame(counts_pos)

In [ ]:
lower, upper = st.t.interval(alpha=0.95, df=len(counts_df)-1, loc=np.mean(counts_df), scale=st.sem(counts_df)) 

In [ ]:
plt.plot(np.arange(0,20,1), counts_df.mean(axis=0).values)
plt.fill_between(np.arange(0,20,1), lower, upper, alpha = 0.5)
plt.xlim(0,19)
plt.ylim(0,5000)
plt.xticks(np.arange(0,20,1), rotation = 45)
plt.xlabel("HybCycles")
plt.ylabel("~Total Spots/Cell")
plt.show()

# This section is to look at collapsed pseudocolor image

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
def plot_all_pseudocolors(df, img, decoded = None, total_hybs = 20, total_rounds = 5, zmax = 1000):
    
    #separate by barcoding rounds
    hybs = np.arange(0, total_hybs, 1)
    hybs_per_round = int(total_hybs/total_rounds)
    rounds = []
    k=0
    for _ in range(total_rounds):
        rounds.append(tuple(hybs[k:k+hybs_per_round]))
        k += hybs_per_round
        
    #display image   
    fig = px.imshow(
        img,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    
    #add spots
    i=1
    for hybs in rounds:
        df_rounds = df[df.hyb.isin(hybs)]
        fig.add_trace(go.Scattergl(
            x=df_rounds.x,
            y=df_rounds.y,
            mode='markers',
            marker_symbol='diamond',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            name=f"Round{i}"
            )
        )
        i += 1
    if decoded is not None:
        fig.add_trace(go.Scattergl(
            x=decoded.x,
            y=decoded.y,
            mode='markers',
            marker_symbol='circle',
            marker=dict(
                #maxdisplayed=1000,
                size=8
                ),
            name="Decoded"
            )
        )
        i += 1
    
    fig.show()

In [ ]:
df = pd.read_csv("/groups/CaiLab/personal/Lex/raw/230608_4k_inv_5bs/pyfish_tools/output/dots_detected/Channel_All/Pos0/noise_removed_locations_z_2.csv", index_col=0)
decoded = pd.read_csv("/groups/CaiLab/personal/Lex/raw/230608_4k_inv_5bs/pyfish_tools/output/decoded/noise_removed/final_0.7511.25_seed33_heg_svm_p20.0_diff1_fdr10.0/Channel_All/Pos_0/dots_used_undefined_z_2.csv")

In [ ]:
raw_src = "/groups/CaiLab/personal/Lex/raw/230608_4k_inv_5bs/chromatic_aberration/MMStack_Pos0.ome.tif"
tiff = pil_imread(raw_src, swapaxes=False)
img = np.max(tiff, axis=0)

In [ ]:
plot_all_pseudocolors(df, img[-1], decoded = decoded, total_hybs = 20, total_rounds = 5, zmax = 1000)